# Record store data
## DataQuest project

This notebook is part of the Dataquest.io curriculum.

#### Scenario
The fictitious company Chinook is a record store and has several business inquiries that require data to base decisions on. These questions are:

1. Which genres sell best in the USA?
2. Which sales support agent performs best in terms of revenue brought in?
3. Which countries bring in the most revenue?
4. Whether consumers purchase single tracks or whole albums.

#### Dataset
A sqlite database of record sales.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view")

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Most sold genres in the USA

There are 4 artists who have brought out 1 new album each, 3 of which will be added to the Chinook store and advertised for. We need to determine which genres sell the best in the USA so that we can determine which of these 4 albums are worth our advertising investment.

| Artist Name          | Genre   |
|----------------------|---------|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

In [5]:
%%sql

WITH overall AS (
    SELECT
        i.billing_country AS country,
        SUM(il.quantity) AS total
    FROM invoice_line AS il
    LEFT JOIN invoice AS i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = "USA"
    ), 
    
    genres AS (
    SELECT 
        g.name,
        SUM(il.quantity) AS tracks_sold_abs,
        i.billing_country AS country
    FROM invoice_line AS il
    LEFT JOIN track AS t ON il.track_id = t.track_id
    LEFT JOIN genre AS g ON t.genre_id = g.genre_id
    LEFT JOIN invoice AS i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = "USA"
    GROUP BY g.name
    ),
    result AS (
    SELECT 
        g.name AS genre,
        g.tracks_sold_abs,
        CAST(g.tracks_sold_abs AS FLOAT) / CAST(o.total AS FLOAT) * 100 AS tracks_sold_pct
    FROM genres AS g
    LEFT JOIN overall AS o ON g.country = o.country
    )
    
SELECT *
FROM result
WHERE genre LIKE '%Punk%' OR genre LIKE '%Hip Hop%'  OR genre LIKE '%Blues%' OR genre LIKE '%Pop%'

 * sqlite:///chinook.db
Done.


genre,tracks_sold_abs,tracks_sold_pct
Alternative & Punk,130,12.369172216936251
Blues,36,3.425309229305423
Hip Hop/Rap,20,1.9029495718363463
Pop,22,2.093244529019981


The Punk genre is lumped in together with Alternative, which may skew the results somewhat. It is by far the most successful genre however. Hiphop loses out to Pop just by a very low margin as the worst performer.

It would make sense to spend the advertising budget according to these percentages, i.e. the majority going to the Alternative & Punk album.

### Performance of sales support agents

Each customer of the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. The company wants to analyse the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [43]:
%%sql

SELECT
    e.first_name || " " || e.last_name AS employee_name,
    e.title,
    e.hire_date,
    SUM(i.total) AS total_dollars
FROM employee AS e
LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
WHERE title = "Sales Support Agent"
GROUP BY e.employee_id

 * sqlite:///chinook.db
Done.


employee_name,title,hire_date,total_dollars
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.510000000004
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0000000000032
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.9200000000028


While Jane Peacock has the highest dollar amount, she has also worked at the company the longest. Steven Johnson, who joined 6 months later, seems to be remarkably close in dollar amount. There is no data on how many hours a week each employee works though, which would also influence results. 

The next best thing is to take the length of their employment in number of days and divide the total_dollars by that amount, to get an estimate of how much revenue sales support agents are bringing in. This will not give us a true absolute value, as this uses calendar days rather than working days. 

In [44]:
%%sql

SELECT
    e.first_name || " " || e.last_name AS employee_name,
    e.title,
    e.hire_date,
    julianday('2020-12-31') - julianday(e.hire_date) AS days_employed,
    SUM(i.total) AS total_dollars,
    SUM(i.total) / (julianday('2020-12-31') - julianday(e.hire_date)) AS dollars_per_day
FROM employee AS e
LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
WHERE title = "Sales Support Agent"
GROUP BY e.employee_id

 * sqlite:///chinook.db
Done.


employee_name,title,hire_date,days_employed,total_dollars,dollars_per_day
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1370.0,1731.510000000004,1.2638759124087622
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1338.0,1584.0000000000032,1.1838565022421548
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1171.0,1393.9200000000028,1.190367207514947


It seems that even when corrected for her working there the longest, Jane Peacock still brings in the most revenue.

### Sales by country

Per country, the company wants to know:
- total number of customers  
- total value of sales  
- average value of sales per customer  
- average order value  

For countries where there is only 1 customer, these customers will be grouped in the category *Other*.

In [8]:
%%sql

WITH 
    other AS (
        SELECT
            c.country,
            COUNT(DISTINCT c.customer_id) AS customers,
            c.customer_id,
            SUM(i.total) AS total_value,
            CAST(SUM(i.total) AS FLOAT) / CAST(COUNT(i.total) AS FLOAT) AS avg_value
        FROM customer AS c
        LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
        GROUP BY c.country
        HAVING customers = 1
    ),
    other_result AS (
        SELECT
            "Other" AS country,
            SUM(customers) AS total_customers,
            AVG(total_value) AS avg_value_customer,
            AVG(avg_value) AS avg_order_value,
            2 AS rank
        FROM other
    ),
    countries AS (
        SELECT
            c.country,
            COUNT(DISTINCT c.customer_id) AS customers,
            SUM(i.total) AS total_value,
            CAST(SUM(i.total) AS FLOAT) / CAST(COUNT(i.total) AS FLOAT) AS avg_value,
            1 AS rank
        FROM customer AS c
        LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
        GROUP BY c.country
        HAVING customers > 1
    ),
    ranking AS (
        SELECT *
        FROM countries
        UNION
        SELECT *
        FROM other_result
        ORDER BY rank
    )
    
SELECT 
    country,
    customers,
    ROUND(total_value,2) AS total_value,
    ROUND(total_value / CAST(customers AS FLOAT),2) AS avg_customer_value,
    ROUND(avg_value,2) AS avg_value 
FROM ranking

 * sqlite:///chinook.db
Done.


country,customers,total_value,avg_customer_value,avg_value
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
Czech Republic,2,273.24,136.62,9.11
France,5,389.07,77.81,7.78
Germany,4,334.62,83.65,8.16
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.56,6.38
USA,13,1040.49,80.04,7.94
United Kingdom,3,245.52,81.84,8.77
Other,15,73.0,4.87,7.45


### Purchases: whole album vs individual tracks

The company wants to know how many purchases are of a whole album vs individual tracks to inform the purchasing strategy.

There will be some edge cases that will reduce the accuracy of the analysis somewhat, such as albums which consist of only 1 track or customers buying each track on an album individually. The company has already confirmed that these cases are rare and thus of not too much influence.

In [9]:
%%sql
WITH
    purchased AS (
        SELECT 
            il.invoice_id, 
            t.album_id,
            COUNT(DISTINCT il.track_id) AS tracks
        FROM invoice_line AS il
        LEFT JOIN track AS t ON t.track_id = il.track_id
        GROUP BY 1,2
        ORDER BY 1,2
    ),
    
    albums AS (
        SELECT
            album_id,
            COUNT(DISTINCT track_id) AS tracks
        FROM track
        GROUP BY 1
        ORDER BY 1
    ),

    overall AS (
        SELECT 
            p.invoice_id,
            p.album_id AS p_album_id,
            p.tracks AS p_tracks,
            a.album_id AS a_album_id,
            a.tracks AS a_tracks
        FROM purchased AS p
        LEFT JOIN albums AS a ON p.album_id = a.album_id
    ),
    result AS (
        SELECT 
            o.*,
            CASE
                WHEN p_tracks = a_tracks THEN 'YES'
                WHEN p_tracks > a_tracks THEN 'AWKWARD'
                WHEN p_tracks < a_tracks THEN 'NO'
                END AS album_purchase
        FROM overall AS o
        ORDER BY album_purchase ASC
    ),
    sorted_result AS (
        SELECT *
        FROM result
        GROUP BY invoice_id
    )
      
SELECT
    sr.album_purchase,
    COUNT(sr.invoice_id) AS num_invoices,
    ROUND(CAST(COUNT(sr.invoice_id) AS FLOAT) / CAST((SELECT COUNT(*) FROM sorted_result) AS FLOAT) * 100,2) AS pct_invoices
FROM sorted_result AS sr
GROUP BY 1

 * sqlite:///chinook.db
Done.


album_purchase,num_invoices,pct_invoices
NO,500,81.43
YES,114,18.57


Looking through the data, it became apparent that multiple customers bought entire albums by buying the tracks individually. Some of these orders were identified because customers can either (a) buy an entire album or (b) buy individual tracks, which can amount to an album (plus potentially additional songs). In the case where additional songs were added, these were identified as orders where customers ordered individual songs. In the case where customers bought exactly all the songs on one album, these were always identified as an album purchase.

Acknowledging this caveat in the analysis, about 81% out of all orders were done as individual songs and not as an album purchase.